In [ ]:
!pip install nibabel pillow numpy

In [14]:
import nibabel as nib
import numpy as np
from PIL import Image
import os
import cv2 # Assicurati che OpenCV sia importato

def save_nii_slices_as_jpeg(nii_filepath, output_folder, slice_axis, file_prefix_for_jpeg, specific_slice_indices):
    """
    Carica un file NIfTI, estrae le fette 2D specificate, filtra per mantenere
    solo i pixel con etichetta 1 (impostandoli a intensità 85),
    RUOTA LA MASCHERA DI 90 GRADI IN SENSO ANTIORARIO,
    e le salva come file JPEG.

    Args:
        nii_filepath (str): Percorso del file .nii o .nii.gz.
        output_folder (str): Cartella dove salvare le immagini JPEG.
        slice_axis (int): L'asse lungo il quale effettuare lo slicing (0, 1, o 2).
        file_prefix_for_jpeg (str): Prefisso per i nomi dei file JPEG salvati (es. "unogt").
        specific_slice_indices (list): Lista degli indici delle fette da estrarre e salvare.
    """
    try:
        # 1. Caricare il file NIfTI
        img = nib.load(nii_filepath)
        print(f"File NIfTI caricato: {nii_filepath}")

        # 2. Ottenere i dati dell'immagine come array NumPy
        data = img.get_fdata() # Restituisce dati come float
        print(f"Shape dei dati dell'immagine: {data.shape}")

        # 3. Creare la cartella di output se non esiste
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print(f"Cartella di output creata: {output_folder}")

        # 4. Verificare che l'asse di slicing sia valido
        if not (0 <= slice_axis < data.ndim):
            print(f"Errore: Asse di slicing {slice_axis} non valido per dati con {data.ndim} dimensioni.")
            return

        # 5. Iterare attraverso gli indici delle fette specificate
        slices_salvate = 0
        target_label_value = 1  # Etichetta dell'oggetto da isolare
        output_intensity = 85   # Intensità da usare per l'oggetto target

        for slice_idx in specific_slice_indices:
            if not (0 <= slice_idx < data.shape[slice_axis]):
                print(f"Attenzione: Indice di fetta {slice_idx} fuori dai limiti per l'asse {slice_axis} (dimensione: {data.shape[slice_axis]}) per il file {nii_filepath}. Salto questa fetta.")
                continue

            # Estrarre la fetta 2D
            if slice_axis == 0:
                slice_2d_float = data[slice_idx, :, :]
            elif slice_axis == 1:
                slice_2d_float = data[:, slice_idx, :]
            elif slice_axis == 2:
                slice_2d_float = data[:, :, slice_idx]
            else:
                print("Errore: Gestione per più di 3 dimensioni non implementata.")
                return

            # 6. Processare la fetta:
            slice_labels_int = np.round(slice_2d_float).astype(np.uint8)

            # 7. Filtrare per mantenere solo l'oggetto con l'etichetta target
            filtered_slice_data = np.zeros_like(slice_labels_int, dtype=np.uint8)
            mask = (slice_labels_int == target_label_value)
            filtered_slice_data[mask] = output_intensity

            # *** MODIFICA CHIAVE: Ruota la maschera di 90 gradi in senso antiorario ***
            # cv2.ROTATE_90_COUNTERCLOCKWISE ruota di 90 gradi in senso antiorario
            rotated_slice_data = cv2.rotate(filtered_slice_data, cv2.ROTATE_90_COUNTERCLOCKWISE)

            # 8. Convertire l'array NumPy ruotato in un'immagine PIL
            pil_image = Image.fromarray(rotated_slice_data) # Usa i dati ruotati

            # 9. Salvare l'immagine come JPEG
            filename = f"{file_prefix_for_jpeg}{slice_idx}.jpeg"
            filepath = os.path.join(output_folder, filename)
            pil_image.save(filepath)
            slices_salvate += 1

        if slices_salvate > 0:
            print(f"{slices_salvate} fette filtrate e ruotate sono state salvate nella cartella '{output_folder}'.")
        else:
            print(f"Nessuna fetta specificata è stata salvata per {nii_filepath} (controllare gli indici e i limiti, o se l'etichetta {target_label_value} era presente).")

    except FileNotFoundError:
        print(f"Errore: File NIfTI non trovato a '{nii_filepath}'.")
    except Exception as e:
        print(f"Si è verificato un errore durante l'elaborazione di {nii_filepath}: {e}")

# --- Configurazione principale per l'elaborazione batch ---
if __name__ == "__main__":
    nii_file_names = [
        "001_left.nii.gz", "002_left.nii.gz", "003_left.nii.gz",
        "008_left.nii.gz", "015_left.nii.gz", "024_left.nii.gz"
        # Aggiungi qui altri nomi di file NIfTI se necessario
    ]
    base_directory = "/content/" # Assicurati che i file NIfTI siano qui
                                 # o modifica questo percorso

    slice_indices_to_extract = []
    slice_indices_to_extract.extend(list(range(90, 100 + 1)))
    slice_indices_to_extract.extend(list(range(140, 150 + 1)))
    slice_indices_to_extract.extend(list(range(160, 170 + 1)))
    slice_indices_to_extract.extend(list(range(190, 200 + 1)))
    slice_indices_to_extract.extend(list(range(220, 230 + 1)))

    print(f"Indici delle fette da estrarre: {sorted(list(set(slice_indices_to_extract)))}")

    jpeg_naming_prefix = "unogt" # Prefisso per i nomi dei file delle maschere
    axis_to_slice = 2 # Assiale (z-axis in molte convenzioni mediche per NIfTI)

    for nii_file_name in nii_file_names:
        input_nifti_file_path = os.path.join(base_directory, nii_file_name)

        # Estrai l'ID numerico dal nome del file (es. "001" da "001_left.nii.gz")
        file_id_str = nii_file_name.split('_')[0]

        try:
            # Formatta il numero della cartella (es. "01", "15")
            folder_number_formatted = f"{int(file_id_str):02d}"
            output_subfolder_name = f"groundtruth{folder_number_formatted}"
            full_output_folder_path = os.path.join(base_directory, output_subfolder_name)

            print(f"\n--- Inizio elaborazione per: {nii_file_name} ---")
            print(f"Cartella di output: {full_output_folder_path}")

            if not os.path.exists(input_nifti_file_path):
                print(f"ATTENZIONE: Il file di input '{input_nifti_file_path}' non esiste. Salto questo file.")
                continue

            save_nii_slices_as_jpeg(
                nii_filepath=input_nifti_file_path,
                output_folder=full_output_folder_path,
                slice_axis=axis_to_slice,
                file_prefix_for_jpeg=jpeg_naming_prefix,
                specific_slice_indices=slice_indices_to_extract
            )
        except ValueError:
            print(f"Errore: Impossibile estrarre un ID numerico valido da '{nii_file_name}'. Salto questo file.")
        except Exception as e:
            print(f"Errore non gestito durante la preparazione per '{nii_file_name}': {e}")

    print("\n--- Elaborazione batch completata. ---")

Indici delle fette da estrarre: [90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230]

--- Inizio elaborazione per: 001_left.nii.gz ---
Cartella di output: /content/groundtruth01
File NIfTI caricato: /content/001_left.nii.gz
Shape dei dati dell'immagine: (400, 510, 243)
55 fette filtrate e ruotate sono state salvate nella cartella '/content/groundtruth01'.

--- Inizio elaborazione per: 002_left.nii.gz ---
Cartella di output: /content/groundtruth02
File NIfTI caricato: /content/002_left.nii.gz
Shape dei dati dell'immagine: (400, 510, 243)
55 fette filtrate e ruotate sono state salvate nella cartella '/content/groundtruth02'.

--- Inizio elaborazione per: 003_left.nii.gz ---
Cartella di output: /content/groundtruth03
File NIfTI caricato: /content/003_left.nii.gz
Shape dei dati de

In [16]:
!zip -r groundtruth01.zip /content/groundtruth01
!zip -r groundtruth02.zip /content/groundtruth02
!zip -r groundtruth03.zip /content/groundtruth03
!zip -r groundtruth08.zip /content/groundtruth08
!zip -r groundtruth15.zip /content/groundtruth15
!zip -r groundtruth24.zip /content/groundtruth24

updating: content/groundtruth01/ (stored 0%)
updating: content/groundtruth01/unogt227.jpeg (deflated 87%)
updating: content/groundtruth01/unogt193.jpeg (deflated 56%)
updating: content/groundtruth01/unogt140.jpeg (deflated 50%)
updating: content/groundtruth01/unogt146.jpeg (deflated 51%)
updating: content/groundtruth01/unogt149.jpeg (deflated 50%)
updating: content/groundtruth01/unogt169.jpeg (deflated 48%)
updating: content/groundtruth01/unogt221.jpeg (deflated 87%)
updating: content/groundtruth01/unogt228.jpeg (deflated 87%)
updating: content/groundtruth01/unogt197.jpeg (deflated 57%)
updating: content/groundtruth01/unogt145.jpeg (deflated 50%)
updating: content/groundtruth01/unogt99.jpeg (deflated 54%)
updating: content/groundtruth01/unogt166.jpeg (deflated 49%)
updating: content/groundtruth01/unogt195.jpeg (deflated 55%)
updating: content/groundtruth01/unogt191.jpeg (deflated 54%)
updating: content/groundtruth01/unogt220.jpeg (deflated 87%)
updating: content/groundtruth01/unogt224.